In [ ]:
pip install kafka-python

In [23]:
import os
import pandas as pd
from kafka import KafkaProducer
from time import sleep
from json import dumps
from time import time
import json
from dotenv import load_dotenv

In [24]:
load_dotenv()

True

In [25]:
KAFKA_SERVER = os.getenv('KAFKA_SERVER')
KAFKA_PORT = os.getenv('KAFKA_PORT')

In [26]:
producer = KafkaProducer(bootstrap_servers=[f"{KAFKA_SERVER}:{KAFKA_PORT}"], 
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'),
                         batch_size=32768,
                         linger_ms=20,
                         compression_type='lz4')

In [27]:
df = pd.read_csv("data/indexProcessed.csv")

In [28]:
num_messages = 5000

In [29]:
start_time = time()

In [30]:
for _ in range(num_messages):
    dict_stock = df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_test', value=dict_stock)

In [31]:
producer.flush()

In [32]:
end_time = time()

In [33]:
total_time = end_time - start_time
print(f"Sent {num_messages} messages in {total_time:.2f} seconds")
print(f"Average throughput: {num_messages / total_time:.2f} messages/second")

Sent 5000 messages in 16.78 seconds
Average throughput: 297.95 messages/second
